In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.yahoo as yfin_data

from datetime import datetime

In [31]:
start = datetime (2010, 1, 1)
end = datetime (2019, 12, 31)

df = yfin_data.daily.YahooDailyReader ('AAPL')
#df = yfin_data.daily.YahooDailyReader ('AAPL', start = "JAN-01-2010", end = "DEC-31-2019", retry_count=3, pause=0.5, session=None, adjust_price=False, ret_index=False, chunksize=1, interval='d', get_actions=False, adjust_dividends=True).read()
print (df)